# Работаем с текстами на Python: кодировки, нормализация, чистка #

# https://habr.com/ru/post/579868/ #

### Зачем эта статья? ###
Об обработке текстов на естественном языке сейчас знают все. Все хоть раз пробовали задавать вопрос Сири или Алисе, пользовались Grammarly (это не реклама), пробовали генераторы стихов, текстов... или просто вводили запрос в Google. Да, вот так просто. На самом деле Google понимает, что вы от него хотите, благодаря штукам, которые умеют обрабатывать и анализировать естественную речь в вашем запросе.

При анализе текста мы можем столкнуться с ситуациями, когда текст содержит специфические символы, которые необходимо проанализировать наравне с "простым текстом" (взять даже наши горячо любимые вставки на французском из "Война и мир") или формулы, например. В таком случае обработка текста может усложниться.

Вы можете заметить, что если ввести в поисковую строку запрос с символами с ударением (так называемый модифицирующий акут), к примеру "ó", поисковая система может показать результаты, содержащие слова из вашего запроса, символы с ударением уже выглядят как обычные символы.

Запрос содержит символ с модифицирующим акутом, однако во втором результате мы можем заметить, что выделено найденное слово из запроса, только вот оно не содержит вышеупомянутый символ, просто букву "о".

Конечно, уже есть много готовых инструментов, которые довольно неплохо справляются с обработкой текстов и могут делать разные крутые вещи, но я не об этом хочу вам поведать. Я не буду рассказывать про nltk, стемминг, лемматизацию и т.п. Я хочу опуститься на несколько ступенек ниже и обсудить некоторые тонкости кодировок, байтов, их обработки.

### Откуда взялась статья? ###
Одним из важных составляющих в области ИИ является обработка текстов на естественном языке. В процессе изучения данной тематики я начал задавать себе вопросы, которые в конечном итоге привели меня к изучению кодировок, представлению текстов в памяти, как они преобразуются, приводятся к нормальной форме. Я плохо понимал эту тему в начале, потребовалось немало времени и мозгового ресурса, чтобы понять, принять и запомнить некоторые вещи. Написанием данной статьи я хочу облегчить жизнь людям, которые столкнутся с необходимостью чтения и обработки текстов на Python и самому закрепить изученное. А некоторыми полезными поинтами своего изучения я постараюсь поделиться в данной статье.

Важная ремарка: я не являюсь специалистом в области обработки текстов. Изложенный материал является результатом исключительно любительского изучения.


### Проблема чтения файлов ###
Допустим, у нас есть файл с текстом. Нам нужно этот текст прочитать. Казалось бы, пиши себе такой вот скрипт для чтения из файла да и радуйся:

In [8]:
with open("ITDO.txt", "r") as file:
    content = file.read()

print(content)

яю1 .   K  ?@>2>48<  B5AB8@>20=85  :>48@>2>:


В файле содержится вот такое вот изречение:
`pitón`

что переводится с испанского как питон. Однако консоль OC Windows 10 покажет нам немного другой результат:

`C:\my\habr\TextsInPython> python .\script1.py
pitÃ³n`

Сейчас мы разберёмся, что именно пошло не так и по какой причине.

### Кодировка ###
Думаю, это не будет сюрпризом, если я скажу, что любой символ, который заносится в память компьютера, хранится в виде числа, а не в виде литерала. Это число определяется как идентификатор или кодовая позиция символа. Кодировка определяет, какое именно число будет ассоциировано с символом.

Предположим, у нас есть некоторый файл с неизвестным содержимым, и нам нужно его прочитать, однако мы не знаем, какая у файла кодировка. Попробуем декодировать содержимое файла.

`C:\my\habr\TextsInPython> python .\script2.py
ÿþ<♦8♦@♦`

Очень интересно, ничего непонятно. По умолчанию Python использует кодировку utf-8, но видимо запись в файл происходила не с её помощью. Здесь нам придёт на помощь дополнительный параметр функции open - параметр encoding, который позволяет указать конкретную кодировку, в которой следует прочитать файл (или записывать в него). Попробуем перебрать несколько кодировок и найти подходящую.

In [9]:
codecs = ["cp1252", "cp437", "utf-16be", "utf-16"]

for codec in codecs:
    with open("ITDO.txt", "r", encoding=codec) as file:
        text = file.read()
    print(codec.rjust(12), "|", text)

      cp1252 | ÿþ1 .   K  ?@>2>48<  B5AB8@>20=85  :>48@>2>:
       cp437 |  ■1 .   K  ?@>2>48<  B5AB8@>20=85  :>48@>2>:
    utf-16be | ￾㄀⸀ ᰄ䬄 㼄䀄㸄㈄㸄㐄㠄㰄 䈄㔄䄄䈄㠄䀄㸄㈄〄㴄㠄㔄 㨄㸄㐄㠄䀄㸄㈄㸄㨄
      utf-16 | 1. Мы проводим тестирование кодировок


Разные кодировки расшифровывают байты из файла по-разному, то есть разным кодовым позициям могут соотвествовать разные символы. Пример примитивный, несложно догадаться, что истинная кодировка файла - это utf-16.

`Важный поинт:` при записи и чтении из файлов следует указывать конкретную кодировку, это позволит избежать путаницы в дальнейшем.

### Ошибки, связанные с кодировками
При возникновении ошибки, связанной с кодировками, интерпретатор выдаст одно из следующих исключений:

- UnicodeError. Это общее исключение для ошибок кодировки.
- UnicodeDecodeError. Данное исключение возбуждается, если встречается кодовая позиция, которая отсутствует в кодировке.
- UnicodeEncodeError. А это исключение возбуждается, когда символ, который необходимо закодировать, незнаком для кодировки.

ошибка:
`UnicodeDecodeError: 'ascii' codec can't decode byte 0xc3 in position 3: ordinal not in range(128)`

Кодировка ASCII не поддерживает никакой алфавит, кроме английского. Поэтому декодирование символа "ó" вызывает у ASCII сложности. Однако Python всемогущ и есть механизм, который позволяет обработать ошибки кодировок. Это дополнительный параметр методов encode и decode - параметр errors. Он может принимать следующие значения:

Для обеих функций:

- `replace` - Несоотвествующие символы заменяются на символ ?

- `xmlcharrefreplace` - Несоответствующие символы заменяются на соответсвующие значения XML.

- `backslashreplace` - Несоответствующие символы заменяются на определённые последовательности с обратным слэшем.

- `namereplace` - Несоответствующие символы заменяются на имена этих символов, которые берутся из базы данных Unicode.

Также отдельно выделены значения `surrogatepass` и `surrogateescape.`

Приведём пример использования таких обработчиков:

```
>>> text = "pitón"
>>> text.encode("ascii", errors="ignore")
b'pitn'
>>> text.encode("ascii", errors="replace")
b'pit?n'
>>> text.encode("ascii", errors="xmlcharrefreplace")
b'pit&#243;n'
>>> text.encode("ascii", errors="backslashreplace")
b'pit\\xf3n'
>>> text.encode("ascii", errors="namereplace")
b'pit\\N{LATIN SMALL LETTER O WITH ACUTE}n'
```

`Важный поинт:` если в текстах могут встретиться неожиданные для кодировки символы, во избежание возбуждения исключений можно использовать обработчики.

Важный поинт: если в текстах могут встретиться неожиданные для кодировки символы, во избежание возбуждения исключений можно использовать обработчики.

### Cворачивание регистра
Сворачивание регистра - это попытка унифицировать текст любого представления к канонической форме. Например, приведение всего текста в нижний регистр. Также над текстом производятся некоторые преобразования (например, немецкая «эсцет» - «ß» - преобразуется в «ss»). В Python 3.3 появился метод str.casefold(), который как раз выполняет сворачивание регистра. Если текст содержит только символы кодировки latin1, результат применения этого метода будет аналогичен методу str.lower().

И по классике приведём пример:

```
>>> text = "Die größte Stadt der Welt liegt in China"
>>> text.casefold()
'die grösste stadt der welt liegt in china'
```

В результате применённый метод не только привёл весь текст к нижнему регистру, но и преобразовал специфический немецкий символ.

Важный поинт: привести текст можно не только методом `str.lower()`, но и методом `str.casefold()`, который может выполнить дополнительные преобразования текста.


### Нормализация
Нормализация - это полноценное приведение текста к единому представлению.

Чтобы обозначить важность нормализации, приведём простой пример:

```
letter1 = "µ"
letter2 = "μ"
```

Внешне два этих символа выглядят абсолютно одинаково. Однако если мы попытаемся вывести имена этих символов, как их видит интерпретатор Python'a, результат нас порядком удивит.

В Python есть отличный встроенный модуль, который содержит данные о символах Unicode, их имена, являются ли они цифрамии и т.п. (методы по типу str.isdigit() берут информацию из этих данных). Воспользуемся данным модулем, чтобы вывести имена символов, исходя из информации, которая содержится в базе данных Unicode.

In [10]:
import unicodedata

letter1 = "µ"
letter2 = "μ"
print(unicodedata.name(letter1))
print(unicodedata.name(letter2))

MICRO SIGN
GREEK SMALL LETTER MU


Итак, интерпретатор Python'a видит эти символы как два разных, но в стандарте Unicode они имеют одинаковое отображение.Такие символы называют каноническими эквивалентами. Приложения будут считать два этих символа одинаковыми, но не интерпретатор.

Посмотрим на ещё один пример:

```
>>> s1 = 'café'
>>> s2 = 'cafe\u0301'
>>> s1, s2
('café', 'café')
>>> s1 == s2
False
>>> len(s1), len(s2)
(4, 5)
```

Данные символы также будут являться каноническими эквивалентами. Из примера мы видим, что символ "é" в стандарте Unicodeможет быть представлен двумя способами, которые к тому же имеют разную длину. Символ "é" может быть представлен одним или двумя байтами.

Решением таких конфликтов занимается нормализация. Она реализована в Python в функции unicodedata.normalize.Первым аргумент является так называемая форма нормализации - нормализации строк Unicode, которые позволяют определить, эквивалентны ли какие-либо две строки Unicode друг другу. Всего предлагается четыре формы:

1) Normalization Form D (NFD) - Canonical Decomposition

2) Normalization Form C (NFC) - Canonical Decomposition, следующая за Canonical Composition

3) Normalization Form KD (NFKD) - Compatibility Decomposition

4) Normalization Form KC (NFKC) - Compatibility Decomposition, следующая за Canonical Composition

Разберём каждую форму немного подробнее.

#### NFC

При указании данной формы нормализации происходит каноническая композиция (как, собственно, и гласит название) кодовых позиций с целью получения самой короткой эквивалентной строки.

```>>> unicodedata.normalize("NFC", s1), unicodedata.normalize("NFC", s2)
('café', 'café')
>>> len(unicodedata.normalize("NFC", s1)), len(unicodedata.normalize("NFC", s2))
(4, 4)
>>> unicodedata.normalize("NFC", s1) == unicodedata.normalize("NFC", s2)
True
>>> len(unicodedata.normalize("NFC", s1)) == len(unicodedata.normalize("NFC", s2))
True
```
Итак, нормализация обеих строк внешне их не изменила, однако длина строки s2 стала равной 4 (т.е. на один байт меньше). Была произведена композиция байтов e\u0301, которые являлись отображением "é". Данная последовательность была заменена на минимальное представление символа, т.е. теперь представление этого символа для интерпретатора выглядит как в строке s1. Как результат, мы видим, что длина нормализованных строк стала равной, и сами строки также стали равны.

#### NFD

С этой формой ситуация аналогичная, только происходит декомпозиция байтов, т.е. разложение символа на несколько байт.
```
>>> unicodedata.normalize("NFD", s1), unicodedata.normalize("NFD", s2)
('café', 'café')
>>> len(unicodedata.normalize("NFD", s1)), len(unicodedata.normalize("NFD", s2))
(5, 5)
>>> unicodedata.normalize("NFD", s1) == unicodedata.normalize("NFD", s2)
True
>>> len(unicodedata.normalize("NFD", s1)) == len(unicodedata.normalize("NFD", s2))
True
```

Здесь мы видим, что длина строки s1 увеличилась на один байт. Думаю, уже несложно догадаться, почему.

На данном этапе настал момент ввести понятие символа совместимости. Символы совместимости (compatibility characters) были введены в Unicode ради совместимости с другими стандартами, в частности, стандарты, которые предшествовали Unicode. Это означает, что некоторые символы могут встречаться в стандарте несколько раз. Мы уже могли наблюдать это явление в начале этого раздела на примере с символом "мю". Он считается символом совместимости.

#### NFKC и NFKD

При данных формах нормализации символы совместимости заменяются на его более предпочтительное представление, что также называется совместимой декомпозицией. Однако при данных формах нормализации может быть потеряно форматирование.

Немного модифицируем наш пример из начала раздела. Выведем кодовые позиции символов до и после нормализации:

In [11]:
import unicodedata

letter1 = "µ"
letter2 = "μ"
print("Before normalizing:", ord(letter1), ord(letter2))
letter1 = unicodedata.normalize("NFKC", letter1)
letter2 = unicodedata.normalize("NFKC", letter2)
print("After normalizing:", ord(letter1), ord(letter2))

Before normalizing: 181 956
After normalizing: 956 956


Итак, мы видим, что первый символ (который являлся знаком "микро") был заменён на греческую "мю", т.е. более предпочтительное представление символа. Таким образом, если необходимо, например, провести частотный анализ текста, формы нормализации, которые затрагивают символы совместимости, могут помочь с этим, приводя символы совместимости к единому представлению.

> Важный поинт: нормализация может очень помочь для поиска валидных документов или индексирования текста. Если вы занимаетесь разработкой таких систем, не стоит сбрасывать алгоритмы нормализации со счетов.

### Дополнительные материалы: что использовалось в статье и что почитать по теме
"Fluent Python", Лучано Ромальо

В этой книге целая глава посвящена изучению строк, байтов и Unicode (Глава 4. Тексты и байты). Она есть на русском и английском языках, но в русском переводе допущено немало ошибок, так что открывайте русский вариант на свой страх и риск. Материал статьи в большей степени опирается на данную книгу. Некоторые примеры также взяты оттуда.

### Документация для Unicode на официальном сайте Python

Куда ж без неё, родимой. Там тоже можно найти немало полезной информации, если вам понадобится работать с текстами и делать больше, чем просто считывание из файла. Хотя в некоторых случаях и на этом можно споткнуться.

### Unicode® Standard Annex

Это части стандарта Unicode, которые выложены в открытый доступ в виде отдельных статей.